In [117]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from math import *
import scipy.stats
from pandas import *
#import scipy.integrate as integrate
from scipy.integrate import quad
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from mpl_toolkits import mplot3d

import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.graph_objects as go



In [118]:
def jump_distribution_full(x_value,mu,sigma_jump,step_x):
    lower = x_value - step_x/2.0
    upper = x_value + step_x/2.0
    #print(lower,upper)
    def normal_distribution_function(x):
        value = scipy.stats.norm.pdf(x,mu,sigma_jump)
        if sigma_jump == 0.0:
            return 0.0
        else:
            return value
    res, err = quad(normal_distribution_function, lower, upper)
    return res  #output is a vector of the average densities around the regions of each value in the vector x


In [364]:
x0 = -10.0
xn = 10.0
xsteps = 1000
dx = (xn-x0)/xsteps
x = np.arange(x0, xn+dx, dx) #last x value is x = 99 
#print(x,len(x),dx)

In [365]:
t0 = 0.0
tn = 1.0
tsteps = 100
dt = (tn-t0)/tsteps
t = np.arange(t0, tn+dt, dt) #last time value is t = 999.9
r = dt / (dx**2) # ensure r < 1
print(dt)
#print(t,r,len(t),tsteps,dt)

0.01


In [366]:
def a_coeff_rs(x_value, sigma, kappa, theta, jump_rate,q_current):
    coefficient = jump_rate + (sigma**2)/(dx**2) + np.sum(q_current)
    return coefficient

def b_coeff_rs(x_value, sigma, kappa, theta, jump_rate):
    #print(sigma,kappa,theta,jump_rate)
    coefficient_2 = 0.5*(sigma**2)*(1/dx**2) + 0.5*(1/dx)*kappa*(theta-x_value)
    #print(coefficient_2)
    return coefficient_2

def c_coeff_rs(x_value, sigma, kappa, theta, jump_rate):
    coefficient_3 = 0.5*(sigma**2)*(1/dx**2) - 0.5*(1/dx)*kappa*(theta-x_value)
    return coefficient_3   

In [367]:
#M_1 = M_matrix(x,0.1,0.7,1.0,1.0)
#M_inv = np.linalg.inv(M_1)
#DataFrame(M_1)

In [368]:
def M_matrix_rs(x,sigma,kappa, theta, jump_rate, q_current):
    M = np.zeros(shape=(len(x),len(x)))
    for i in range(0,len(x)):
        if x[i] <= 0.0 or x[i] >= 1.0:
            M[i,i] = 1.0
        else:
            a_coefficient = a_coeff_rs(x[i], sigma, kappa, theta, jump_rate, q_current)
            b_coefficient = b_coeff_rs(x[i], sigma, kappa, theta, jump_rate)
            c_coefficient = c_coeff_rs(x[i], sigma, kappa, theta, jump_rate)
            if a_coefficient < 0 or b_coefficient < 0 or c_coefficient <0:
                print('Error')
            M[i,(i-1):(i+2)] = [-dt*c_coefficient, 1+dt*a_coefficient, -dt*b_coefficient]

    return M

In [369]:
def M_final_rs(x,sigma_vec,kappa_vec, theta_vec, jump_rate_vec, Q):
    
    M_final_matrix = np.zeros(shape=(1,len(x)*len(Q)))
    Q_temp = Q[~np.eye(Q.shape[0],dtype=bool)].reshape(Q.shape[0],-1)
    for i in range(0,len(Q)):
        M_block = M_matrix_rs(x,sigma_vec[i],kappa_vec[i],theta_vec[i],jump_rate_vec[i],Q_temp[i,])
        print(Q_temp)
        #print(sigma_vec[i],kappa_vec[i],theta_vec[i],jump_rate_vec[i],Q_temp[i,])
        #print(DataFrame(M_block))
        for j in range(0,len(Q)):
            X_current = np.eye(len(x))*(-dt*Q[i,j])
            X_current[x <= 0.0] = 0.0
            X_current[x >= 1.0] = 0.0
            if j < i:
                M_block = np.block([X_current, M_block])
            if j > i:
                M_block = np.block([ M_block, X_current]) 
        
        M_final_matrix = np.block([
            [M_final_matrix],
            [M_block]
        ])
        
    return M_final_matrix[1:,:]


In [371]:
#Q = np.array([[-0.8, 0.1, 0.1], 
#    [0.2, -0.6, 0.2],
#    [0,0,-1]])

#Q[~np.eye(Q.shape[0],dtype=bool)].reshape(Q.shape[0],-1)

In [372]:
#Q = np.array([[-0.8, 0.1, 0.1], 
#    [0.2, -0.6, 0.2],
#    [0,0,-1]])

#sigma_vec = np.array([1,1.5,0])
#kappa_vec = np.array([0.5,0.5,0])
#theta_vec = np.array([0.5,0.2,0])
#jump_rate_vec = np.array([0.2,0.2,0])
#trial = M_final_rs(x,sigma_vec,kappa_vec, theta_vec, jump_rate_vec, Q)
#DataFrame(trial[0*len(x):1*len(x),0*len(x):1*len(x)])

In [373]:
def N_matrix_rs(x, jump_rate, N, mu, sigma_jump, step_x):
    N_m = np.zeros(shape=(len(x),len(x)))
    positions = np.arange(int(-N/2+1),int(N/2),1)
    for i in range(0,(len(x))):
        if x[i] > 0.0 and x[i] < 1.0:
            for j in positions: 
                x_jump = step_x*j
                integral = jump_distribution_full(x_jump, mu, sigma_jump, step_x)
                if (i+j) >= 0 and (i+j) <= (len(x)-1):
                    #print(j)
                    N_m[i,i+j] = jump_rate*dt*integral
            N_m[i,i] = N_m[i,i]+1
    return N_m

In [374]:
def N_final_rs(x, jump_rate_vec,N_vec,mu_vec,sigma_jump_vec,step_x):
    
    N_final_matrix = np.zeros(shape=(1,len(x)*len(Q)))
    for i in range(0,len(Q)):
        N_block = N_matrix_rs(x, jump_rate_vec[i],N_vec[i],mu_vec[i],sigma_jump_vec[i],step_x)
        for j in range(0,len(Q)):
            X_current = np.zeros((len(x),len(x)))
            if j < i:
                N_block = np.block([X_current, N_block])
            if j > i:
                N_block = np.block([N_block, X_current]) 
    
        N_final_matrix = np.block([
            [N_final_matrix],
            [N_block]
        ])
    
    return N_final_matrix[1:,:]

In [375]:
#jump_rate_vec = np.array([1.0,1.0,0.0])
##N_vec = np.array([4,4,4])
#mu_vec = np.array([0,0,0])
#sigma_jump_vec = np.array([0.1,0.1,0.0])
#N_try = N_final_rs(x, jump_rate_vec,N_vec,mu_vec,sigma_jump_vec,dx)
#(N_try[2*len(x):3*len(x),2*len(x):3*len(x)])

In [376]:
#N_try = N_matrix(x, 1.0,4,0.0,0.1,dx)
#DataFrame(N_try[0:10, 0:10])

In [377]:
#IC_matrix(x,1,1,1,1,0,1,dx,0,1)

In [378]:
def b_matrix(boundary_conditions_vec):
    b_final_matrix = np.array([0])
    for i in range(0,len(boundary_conditions_vec)):
        b_block = np.zeros(len(x))
        b_block[x>=1.0] = boundary_conditions_vec[i]
        b_final_matrix = np.block([
            b_final_matrix, b_block
        ])
    
    return b_final_matrix[1:]

In [379]:
#boundary_conditions_vec = np.array([1,1,0])
#b_matrix(boundary_conditions_vec)

In [380]:
def phi_matrix_def(boundary_conditions_vec,initial_conditions_vec,x,t):
    phi_final_matrix = np.zeros(shape=(1,len(t)))
    for i in range(0, len(boundary_conditions_vec)):
        phi_block = np.zeros(shape = (len(x),len(t)))
        phi_block[:,0] = initial_conditions_vec[i] * (x>0.0)
        phi_block[len(x)-1,:] = boundary_conditions_vec[i]
        
        phi_final_matrix = np.block([
            [phi_final_matrix],
            [phi_block]
        ])
        
    return phi_final_matrix[1:,:]

In [381]:
#DataFrame(phi_matrix_def(boundary_conditions_vec,initial_conditions_vec,x,t)[51:101,51:101])

In [382]:
#x_block = np.block([x,x,x])
#phi_matrix = np.zeros(shape=(3*len(x),len(t))) 
#phi_matrix[:,0] = 1.0 * (x_block > 0.0)
#phi_matrix[len(x)-1,:] = 1.0
#phi_matrix[2*len(x)-1,:] = 1.0
#phi_matrix[(2*len(x)):(3*len(x)),:] = 0.0
#DataFrame(phi_matrix)
#phi_matrix = np.zeros(shape=(len(x), len(t)))
#phi_matrix[:,0] = 1.0 * (x>0.0)
#phi_matrix[len(x)-1,:] = 1.0

In [383]:
Q = np.array([[-0.5, 0.3, 0.2], 
    [0.3, -0.6, 0.3],   
    [0.0,0.0,-0.0]])
#q_12 = 0.2
#q_13 = 0.1
#q_21 = 0.4
#q_23 = 0.5
sigma_vec = np.array([0.5,0.9,0.0])
kappa_vec = np.array([0.3,0.1,0.0])
theta_vec = np.array([0.7,0.4,0.0])
jump_rate_vec = np.array([1.0,1.0,0.0])
mu_vec = np.array([0.0,-0.3,0.0])
sigma_jump_vec = np.array([0.5,1.0,0.0])
N_vec = np.array([4,6,0])
boundary_conditions_vec = np.array([1.0,1.0,0.0])
initial_conditions_vec = np.array([1.0,1.0,0.0])

#sigma_choice_1 = 10.0
#kappa_choice_1 = 7.0
#theta_choice_1 = 5.0
#sigma_choice_2 = 3.0
#kappa_choice_2 = 4.0
#theta_choice_2 = 10.0
#jump_rate_choice = 2.0
#N_choice = 4
#mu_choice = 0.0
#sigma_jump_choice = 5.0

M = M_final_rs(x,sigma_vec,kappa_vec, theta_vec, jump_rate_vec, Q)
#print(DataFrame(M[13:28,13:28]))
N_mat = N_final_rs(x, jump_rate_vec,N_vec,mu_vec,sigma_jump_vec,dx)
#print(DataFrame(N_mat[13:28, 13:28]))
b = b_matrix(boundary_conditions_vec)
#print(b[26:28])
phi_matrix = phi_matrix_def(boundary_conditions_vec,initial_conditions_vec,x,t)

stability = [sigma_vec[i]**2 >= np.max(np.abs(dx*kappa_vec[i]*(theta_vec[i]-x))) for i in range(0,len(sigma_vec))]
#print(all(stability))
#stability = sigma_vec**2 - np.max(np.abs(dx*kappa_vec[i]*(theta_vec[i]-x)))

if all(stability):
    
    for time in range(1, len(t)): 
        phi_matrix[:,time] = np.linalg.inv(M).dot(N_mat.dot(phi_matrix[:,time-1]) + b)
else:
    print('Stability conditions may not be satisfied')
    

[[0.3 0.2]
 [0.3 0.3]
 [0.  0. ]]
[[0.3 0.2]
 [0.3 0.3]
 [0.  0. ]]
[[0.3 0.2]
 [0.3 0.3]
 [0.  0. ]]


In [384]:
accept_x = np.where((x>=0) & (x<=1))[0]
#accept_x[len(accept_x)-1]
accept_x

array([501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513,
       514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526,
       527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539,
       540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550], dtype=int64)

In [395]:
phi_matrix[1*len(x)+507,] #- phi_matrix[1*len(x)+502,]

array([1.        , 0.87946328, 0.75185689, 0.65006081, 0.57305836,
       0.51414366, 0.46782456, 0.4303833 , 0.39937541, 0.37317004,
       0.35064956, 0.33102575, 0.31372743, 0.29833049, 0.284513  ,
       0.27202587, 0.260673  , 0.25029758, 0.24077252, 0.23199351,
       0.22387395, 0.21634123, 0.20933389, 0.20279948, 0.19669288,
       0.19097499, 0.18561173, 0.18057317, 0.17583294, 0.17136763,
       0.16715637, 0.1631805 , 0.15942323, 0.15586939, 0.15250526,
       0.14931836, 0.1462973 , 0.14343166, 0.14071188, 0.13812916,
       0.13567539, 0.13334307, 0.13112523, 0.12901543, 0.12700765,
       0.1250963 , 0.12327615, 0.1215423 , 0.11989019, 0.11831551,
       0.11681424, 0.1153826 , 0.11401701, 0.11271411, 0.11147076,
       0.11028396, 0.1091509 , 0.10806891, 0.10703548, 0.10604823,
       0.10510491, 0.10420338, 0.10334163, 0.10251774, 0.1017299 ,
       0.10097639, 0.10025558, 0.09956591, 0.09890593, 0.09827422,
       0.09766947, 0.09709042, 0.09653587, 0.09600468, 0.09549

In [388]:
phi_matrix[:,1]

array([0., 0., 0., ..., 0., 0., 0.])

In [142]:
#phi_matrix[len(x)+8,] - phi_matrix[len(x)+7,]

In [396]:
for i in range(0*len(x)+accept_x[0],0*len(x)+accept_x[len(accept_x)-1]):
    print(np.min(phi_matrix[i+1,] - phi_matrix[i,]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [397]:
def survival(space,time):
    return phi_matrix[space, time]

In [400]:
X = 2*len(x)+accept_x#np.arange(len(x),2*len(x),1)
T = np.arange(0,len(t),1)
X, T = np.meshgrid(X, T)
Phi = survival(X,T)

data=go.Surface(z=Phi, x=x0+dx*accept_x, y=t0+dt*T)

layout = go.Layout(scene = dict(
                    xaxis_title='Initial Position',
                    yaxis_title='Maturity',
                    zaxis_title='Survival Probability'))

fig = go.Figure(data=[data], layout=layout)
pyoff.plot(fig)

'temp-plot.html'

In [401]:
def survival_av(space,time):
    return (phi_matrix[space,time] + phi_matrix[1*len(x)+space,time] + phi_matrix[2*len(x)+space,time])/3.0

survival_av(550,0)

0.6666666666666666

In [402]:
X = 0*len(x)+accept_x#np.arange(len(x),2*len(x),1)
T = np.arange(0,len(t),1)
X, T = np.meshgrid(X, T)
Phi = survival_av(X,T)

data=go.Surface(z=Phi, x=x0+dx*accept_x, y=t0+dt*T)

layout = go.Layout(scene = dict(
                    xaxis_title='Initial Position',
                    yaxis_title='Maturity',
                    zaxis_title='Survival Probability'))

fig = go.Figure(data=[data], layout=layout)
pyoff.plot(fig)

'temp-plot.html'

In [405]:
surv = DataFrame(phi_matrix)

In [406]:
surv['initial_pos'] = np.block([x,x,x]) #x0+dx*X[1]

In [407]:
times= DataFrame(t)
time= times[0]

In [408]:
surv.append(time)

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,initial_pos
0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-10.00
1,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.98
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.96
3,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.94
4,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,9.94
3000,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,9.96
3001,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,9.98
3002,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,10.00


In [422]:
surv_df_non_zero = surv[(surv['initial_pos'] >= 0.0) & (surv['initial_pos'] <= 1.0)]
surv_df_non_zero

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,initial_pos
501,1.0,0.433722,0.278545,0.215054,0.180499,0.158187,0.142231,0.130052,0.120330,0.112314,...,0.016802,0.016603,0.016409,0.016220,0.016036,0.015856,0.015681,0.015510,0.015344,0.02
502,1.0,0.676910,0.500401,0.404078,0.345052,0.304911,0.275450,0.252622,0.234226,0.218957,...,0.033064,0.032674,0.032293,0.031922,0.031560,0.031207,0.030863,0.030527,0.030200,0.04
503,1.0,0.813348,0.661725,0.558488,0.487275,0.435651,0.396352,0.365208,0.339729,0.318358,...,0.048821,0.048246,0.047685,0.047139,0.046606,0.046087,0.045581,0.045087,0.044605,0.06
504,1.0,0.889956,0.772993,0.677922,0.604710,0.547924,0.502817,0.466054,0.435392,0.409312,...,0.064102,0.063350,0.062618,0.061904,0.061207,0.060528,0.059866,0.059221,0.058591,0.08
505,1.0,0.933007,0.847144,0.766567,0.698036,0.641203,0.593964,0.554235,0.520346,0.491037,...,0.078937,0.078017,0.077120,0.076246,0.075393,0.074562,0.073751,0.072961,0.072190,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.92
2549,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.94
2550,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.96
2551,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.98


In [423]:
accept_t = [0,10,20,30,40,50,60,70,80,90,100]

In [424]:
times = t[accept_t[1:]]
times

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [425]:
output=surv_df_non_zero.iloc[:,accept_t[1:]]
initial_pos = surv['initial_pos'][(surv['initial_pos'] >= 0.0) & (surv['initial_pos'] <= 1.0)]
output['initial_pos'] = initial_pos
survival_values = round(output,4)
survival_values = survival_values.set_index('initial_pos')
#survival_values.columns = t[accept_t]
#output
survival_values

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.1055,0.0682,0.0508,0.0399,0.0324,0.0269,0.0228,0.0196,0.0172,0.0153
0.04,0.2060,0.1338,0.0997,0.0784,0.0637,0.0529,0.0448,0.0386,0.0339,0.0302
0.06,0.3001,0.1964,0.1466,0.1155,0.0938,0.0780,0.0661,0.0570,0.0500,0.0446
0.08,0.3868,0.2558,0.1915,0.1511,0.1228,0.1022,0.0866,0.0748,0.0657,0.0586
0.10,0.4654,0.3117,0.2343,0.1852,0.1508,0.1255,0.1065,0.0920,0.0808,0.0722
...,...,...,...,...,...,...,...,...,...,...
0.92,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.94,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.96,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [426]:
pd = 1-survival_values
pd

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.8945,0.9318,0.9492,0.9601,0.9676,0.9731,0.9772,0.9804,0.9828,0.9847
0.04,0.7940,0.8662,0.9003,0.9216,0.9363,0.9471,0.9552,0.9614,0.9661,0.9698
0.06,0.6999,0.8036,0.8534,0.8845,0.9062,0.9220,0.9339,0.9430,0.9500,0.9554
0.08,0.6132,0.7442,0.8085,0.8489,0.8772,0.8978,0.9134,0.9252,0.9343,0.9414
0.10,0.5346,0.6883,0.7657,0.8148,0.8492,0.8745,0.8935,0.9080,0.9192,0.9278
...,...,...,...,...,...,...,...,...,...,...
0.92,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
0.94,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
0.96,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [427]:
surv_stage_1 = survival_values.iloc[0:int(survival_values.shape[0]/3),:]
surv_stage_2 = survival_values.iloc[int(survival_values.shape[0]/3):int(2*survival_values.shape[0]/3),:]
surv_stage_3 = survival_values.iloc[int(2*survival_values.shape[0]/3):int(3*survival_values.shape[0]/3),:]

In [433]:
surv_stage_1

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.1055,0.0682,0.0508,0.0399,0.0324,0.0269,0.0228,0.0196,0.0172,0.0153
0.04,0.2060,0.1338,0.0997,0.0784,0.0637,0.0529,0.0448,0.0386,0.0339,0.0302
0.06,0.3001,0.1964,0.1466,0.1155,0.0938,0.0780,0.0661,0.0570,0.0500,0.0446
0.08,0.3868,0.2558,0.1915,0.1511,0.1228,0.1022,0.0866,0.0748,0.0657,0.0586
0.10,0.4654,0.3117,0.2343,0.1852,0.1508,0.1255,0.1065,0.0920,0.0808,0.0722
0.12,0.5355,0.3640,0.2749,0.2177,0.1776,0.1480,0.1257,0.1087,0.0956,0.0854
0.14,0.5972,0.4126,0.3133,0.2488,0.2033,0.1697,0.1443,0.1249,0.1099,0.0983
0.16,0.6506,0.4574,0.3494,0.2783,0.2278,0.1905,0.1622,0.1406,0.1239,0.1110
0.18,0.6962,0.4985,0.3832,0.3064,0.2513,0.2105,0.1796,0.1558,0.1375,0.1233


In [434]:
surv_av = (surv_stage_1 + surv_stage_2 + surv_stage_3)/3.0
surv_av

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.052767,0.033700,0.025033,0.019833,0.016400,0.014000,0.012267,0.010967,0.010000,0.009267
0.04,0.103700,0.066467,0.049400,0.039200,0.032433,0.027667,0.024233,0.021700,0.019833,0.018367
0.06,0.152300,0.098133,0.073067,0.058067,0.048033,0.041067,0.036000,0.032233,0.029433,0.027333
0.08,0.198067,0.128667,0.096033,0.076400,0.063267,0.054133,0.047467,0.042600,0.038933,0.036167
0.10,0.240700,0.157867,0.118200,0.094200,0.078167,0.066900,0.058733,0.052733,0.048233,0.044867
0.12,0.280033,0.185733,0.139600,0.111433,0.092633,0.079367,0.069800,0.062733,0.057467,0.053433
0.14,0.315933,0.212167,0.160167,0.128167,0.106700,0.091600,0.080667,0.072600,0.066533,0.061933
0.16,0.348467,0.237133,0.179867,0.144300,0.120367,0.103533,0.091300,0.082267,0.075500,0.070400
0.18,0.377700,0.260633,0.198667,0.159933,0.133700,0.115200,0.101767,0.091833,0.084400,0.078767


In [442]:
phi_cd = np.array([1.00,0.591667,0.523900,0.470067,0.429700,0.399700,0.377267,0.360367,0.347533,0.337733,0.330200])

In [443]:
def CDS_numerator(r,dt, Phi):
    Phi_without_last = Phi[0:(len(Phi)-1)]
    Phi_without_0 = Phi[1:]
    t = np.arange(dt, 1+dt, dt)
    #print(t)
    discount = np.exp(-r*t)
    surv = Phi_without_last - Phi_without_0
    #print(surv)
    num_vector = discount*surv
    return(sum(num_vector))

In [444]:
def CDS_denominator(r,dt, Phi):
    Phi_without_last = Phi[0:(len(Phi)-1)]
    Phi_without_0 = Phi[1:]
    t = np.arange(dt, 1+dt, dt)
    discount = np.exp(-r*t)
    surv = Phi_without_0 + Phi_without_last
    num_vector = discount*surv
    return(sum(num_vector))

In [445]:
def CDS_spread(r,dt,Phi,R):
    price = (1-R)*CDS_numerator(r,dt,Phi) / 0.5*dt*CDS_denominator(r,dt,Phi)
    return(price)

In [446]:
t = np.arange(0.1, 1+0.1, 0.1)
t

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [447]:
CDS_spread(0.1, 0.1, phi_cd,0.5)

0.5625029541194406